### Extractive Summarization

#### End goal - Forming a sub-convo for each question asked by moderator and the various answers given by the parents.
#### Running a summarizer to capture the info from each such sub-convo.
#### This will be very useful to filter out the unnecessary text at the beginning and end of each conversation and to focus on the 
#### important text for analysing the survey responses.

#### Implementation includes aggregating data at a question level and running through 3 pre-trained models-
#### BERT, XLNET, GPT2
#### Based on preliminary runs and results, individual models gave average results and each missed important aspects.
#### All of them had some common content.
#### So the idea here is to use an Ensemble model like technique to combine the results from each of these individual 'weak' models
#### and to aggregate them to in such a way that more variation is captured


In [2]:
import pandas as pd
import os

In [3]:
from summarizer import Summarizer, TransformerSummarizer

In [4]:
pd.set_option('max_colwidth', 4000)

In [5]:
#focus_groups_convo_data = pd.read_csv("../../vcare/data_preparation/focus_groups_convos.csv")
data_folder_path = r'C:\Users\vhpld\Documents\GitHub\ML-for-Good-Hackathon\vcare\data_preparation'
focus_groups_convo_file = "focus_groups_convos.csv"
focus_groups_convo_data_df = pd.read_csv(os.path.join(data_folder_path, focus_groups_convo_file))

In [4]:
focus_groups_convo_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   focus_group_subtype     69 non-null     object 
 1   focus_group_subtype_id  69 non-null     int64  
 2   doc_no_within_subtype   69 non-null     int64  
 3   question_id             69 non-null     int64  
 4   question_text           69 non-null     object 
 5   parent_num              68 non-null     float64
 6   parent_answer           68 non-null     object 
 7   Unnamed: 7              10 non-null     object 
dtypes: float64(1), int64(3), object(4)
memory usage: 4.4+ KB


In [5]:
focus_groups_convo_data_df.head()

,focus_group_subtype,focus_group_subtype_id,doc_no_within_subtype,question_id,question_text,parent_num,parent_answer,Unnamed: 7
0,gaming_group,1,1,1,"So just starting to think through some of the impact of technology, can we talk a little bit about how your child used technology prior to the pandemic for school purposes?",2.0,"Oh okay. Well, they didn't use it as much. I mean, the teachers would assign some reading or math assignments on I ready and from time to time some homework. But yeah, that was about it. I would monitor how much time they were on Roblox and other things to entertain themselves, but it was easier control them back then because they didn't have to go on as much. I mean, that's what it was.",NaN
1,gaming_group,1,1,1,"So just starting to think through some of the impact of technology, can we talk a little bit about how your child used technology prior to the pandemic for school purposes?",3.0,"I would say the same thing. My daughter was in fifth grade, we would have a couple in Newsela projects, half an hour of homework. But we were in the playground, neighborhood friends, after school projects, after school swimming. A little bit of tablet on the weekends, but there was no internet access and there was no YouTube in my life, there was no Roblox, there was no gaming. So, a very different world.",NaN
2,gaming_group,1,1,1,"So just starting to think through some of the impact of technology, can we talk a little bit about how your child used technology prior to the pandemic for school purposes?",5.0,"I have a similar experience to Parent 3. My high schooler had a lot of his homework that he had to type and word process... I don't know if that's the right word, into Docs, but it wasn't a huge amount. It wasn't doing research, it wasn't hours and hours. My elementary school guy, he didn't have any homework on the computer at all. And my middle schooler had very little, like type this one thing or something like that. It was very little on the computer. And for myself, in terms of my family, I lock the video games so that there was very little access to video games during the week.",NaN
3,gaming_group,1,1,1,"So just starting to think through some of the impact of technology, can we talk a little bit about how your child used technology prior to the pandemic for school purposes?",5.0,"And then on the weekends there was an hour or something like that, it wasn't that much. There was no YouTube watching, like now YouTube consumes our life. My kids didn't even know how to use the features on our TV, the smart features, they didn't know they could access YouTube on it or anything like that. So it was very controlled and very little. I also had my children in a lot of afterschool activities. My older son was in an orchestra, he also was in a band, theater groups, there was a lot of afterschool activity. A lot of playing outside, a lot of going to the park and all that stuff. Yeah, so...",NaN
4,gaming_group,1,1,1,"So just starting to think through some of the impact of technology, can we talk a little bit about how your child used technology prior to the pandemic for school purposes?",4.0,"I can go. My kids probably had more... It sounds like they did more online than, so far, the rest of them. But my daughter in high school definitely had almost all of her homework... Not all of it because she was assigned in school, but she definitely was on doing homework, I feel like, a good amount. Nothing like now, but definitely was using it. My middle schooler also, but not as much. They both did play games or use Instagram, my daughter was really into Instagram, not as much anymore but definitely games. Now it's just like a free-for-all. I mean, they're in their rooms with the door shut, I have no idea what they're doing all day. I just know that they're doing their homework, getting good grades. I think they're on electronics the entire day.",NaN


In [8]:
#function to form sub-convos and required format -
"""
Moderator 1: "Question 1 text"
Parent 2: "Response text"
Parent 1: "Response text"
Parent 5: "Response text"
Parent 3: "Response text"
"""
def get_convo_per_question(group_df):    
    d = {}
    question = group_df['question_text'][0]
    convo_question_text = f'Moderator: {question}'
    convo_text = convo_question_text
    
    for row_index, item in group_df.iterrows():
        parent_ans = item['parent_answer']
        parent_num = str(item['parent_num'])
        convo_parent_text = f' Parent {parent_num}: {parent_ans}'
        convo_text  += convo_parent_text    
    
    return convo_text    

In [42]:
#loading the structured data prepared to capture each reponse per question per parent per group convo
focus_groups_convo_data_grouped_df = focus_groups_convo_data_df.groupby(['focus_group_subtype_id', 'doc_no_within_subtype', 'question_id'])
focus_groups_convo_data_grouped_df.head()

,focus_group_subtype,focus_group_subtype_id,doc_no_within_subtype,question_id,question_text,parent_num,parent_answer,Unnamed: 7
0,gaming_group,1,1,1,"So just starting to think through some of the impact of technology, can we talk a little bit about how your child used technology prior to the pandemic for school purposes?",2.0,"Oh okay. Well, they didn't use it as much. I mean, the teachers would assign some reading or math assignments on I ready and from time to time some homework. But yeah, that was about it. I would monitor how much time they were on Roblox and other things to entertain themselves, but it was easier control them back then because they didn't have to go on as much. I mean, that's what it was.",NaN
1,gaming_group,1,1,1,"So just starting to think through some of the impact of technology, can we talk a little bit about how your child used technology prior to the pandemic for school purposes?",3.0,"I would say the same thing. My daughter was in fifth grade, we would have a couple in Newsela projects, half an hour of homework. But we were in the playground, neighborhood friends, after school projects, after school swimming. A little bit of tablet on the weekends, but there was no internet access and there was no YouTube in my life, there was no Roblox, there was no gaming. So, a very different world.",NaN
2,gaming_group,1,1,1,"So just starting to think through some of the impact of technology, can we talk a little bit about how your child used technology prior to the pandemic for school purposes?",5.0,"I have a similar experience to Parent 3. My high schooler had a lot of his homework that he had to type and word process... I don't know if that's the right word, into Docs, but it wasn't a huge amount. It wasn't doing research, it wasn't hours and hours. My elementary school guy, he didn't have any homework on the computer at all. And my middle schooler had very little, like type this one thing or something like that. It was very little on the computer. And for myself, in terms of my family, I lock the video games so that there was very little access to video games during the week.",NaN
3,gaming_group,1,1,1,"So just starting to think through some of the impact of technology, can we talk a little bit about how your child used technology prior to the pandemic for school purposes?",5.0,"And then on the weekends there was an hour or something like that, it wasn't that much. There was no YouTube watching, like now YouTube consumes our life. My kids didn't even know how to use the features on our TV, the smart features, they didn't know they could access YouTube on it or anything like that. So it was very controlled and very little. I also had my children in a lot of afterschool activities. My older son was in an orchestra, he also was in a band, theater groups, there was a lot of afterschool activity. A lot of playing outside, a lot of going to the park and all that stuff. Yeah, so...",NaN
4,gaming_group,1,1,1,"So just starting to think through some of the impact of technology, can we talk a little bit about how your child used technology prior to the pandemic for school purposes?",4.0,"I can go. My kids probably had more... It sounds like they did more online than, so far, the rest of them. But my daughter in high school definitely had almost all of her homework... Not all of it because she was assigned in school, but she definitely was on doing homework, I feel like, a good amount. Nothing like now, but definitely was using it. My middle schooler also, but not as much. They both did play games or use Instagram, my daughter was really into Instagram, not as much anymore but definitely games. Now it's just like a free-for-all. I mean, they're in their rooms with the door shut, I have no idea what they're doing all day. I just know that they're doing their homework, getting good grades. I think they're on electronics the entire day.",NaN
7,gaming_group,1,1,2,If you can walk me through a little bit how your use of technol

In [43]:
#forming sub-convos
focus_groups_convo_groupedby_doc_question_df = focus_groups_convo_data_grouped_df.apply(lambda x : get_convo_per_question(x))

Moderator: So just starting to think through some of the impact of technology, can we talk a little bit about how your child used technology prior to the pandemic for school purposes? Parent 2.0: Oh okay. Well, they didn't use it as much. I mean, the teachers would assign some reading or math assignments on I ready and from time to time some homework. But yeah, that was about it. I would monitor how much time they were on Roblox and other things to entertain themselves, but it was easier control them back then because they didn't have to go on as much. I mean, that's what it was. Parent 3.0: I would say the same thing. My daughter was in fifth grade, we would have a couple in Newsela projects, half an hour of homework. But we were in the playground, neighborhood friends, after school projects, after school swimming. A little bit of tablet on the weekends, but there was no internet access and there was no YouTube in my life, there was no Roblox, there was no gaming. So, a very different

Moderator: But in terms of technology use for school, how did your child or children use technology for school purposes before the pandemic? Parent 3.0: Hi. My name is Parent 3. I showed up a little late, sorry everyone. So I have two sons, ages 7 and 10. They had very little interaction with electronics before the pandemic and now this is all... I mean, I'm sitting across from my child who's supposed to be doing math work, and he's on Friday Night Funkin' or whatever it is. I don't know what it is, Roblox, whatever. I just see him over there jamming, and that's not math. He's not jamming to math. But this is what I deal with daily. I think they felt they were on vacation. This whole time they've just been on a never-ending vacation and [crosstalk 00:16:04] Parent 2.0: I mean, I feel like my kids are watching a lot of inappropriate YouTube stuff too, stuff that I definitely don't really want them exposed to that much, and I just am working and they know that they're over there on YouTu

In [16]:
focus_groups_convo_groupedby_doc_question_df.head()

,focus_group_subtype_id,doc_no_within_subtype,question_id,0
0,1,1,1,"Moderator: So just starting to think through some of the impact of technology, can we talk a little bit about how your child used technology prior to the pandemic for school purposes?Parent 2.0: Oh okay. Well, they didn't use it as much. I mean, the teachers would assign some reading or math assignments on I ready and from time to time some homework. But yeah, that was about it. I would monitor how much time they were on Roblox and other things to entertain themselves, but it was easier control them back then because they didn't have to go on as much. I mean, that's what it was.Parent 3.0: I would say the same thing. My daughter was in fifth grade, we would have a couple in Newsela projects, half an hour of homework. But we were in the playground, neighborhood friends, after school projects, after school swimming. A little bit of tablet on the weekends, but there was no internet access and there was no YouTube in my life, there was no Roblox, there was no gaming. So, a very different world.Parent 5.0: I have a similar experience to Parent 3. My high schooler had a lot of his homework that he had to type and word process... I don't know if that's the right word, into Docs, but it wasn't a huge amount. It wasn't doing research, it wasn't hours and hours. My elementary school guy, he didn't have any homework on the computer at all. And my middle schooler had very little, like type this one thing or something like that. It was very little on the computer. And for myself, in terms of my family, I lock the video games so that there was very little access to video games during the week.Parent 5.0: And then on the weekends there was an hour or something like that, it wasn't that much. There was no YouTube watching, like now YouTube consumes our life. My kids didn't even know how to use the features on our TV, the smart features, they didn't know they could access YouTube on it or anything like that. So it was very controlled and very little. I also had my children in a lot of afterschool activities. My older son was in an orchestra, he also was in a band, theater groups, there was a lot of afterschool activity. A lot of playing outside, a lot of going to the park and all that stuff. Yeah, so...Parent 4.0: I can go. My kids probably had more... It sounds like they did more online than, so far, the rest of them. But my daughter in high school definitely had almost all of her homework... Not all of it because she was assigned in school, but she definitely was on doing homework, I feel like, a good amount. Nothing like now, but definitely was using it. My middle schooler also, but not as much. They both did play games or use Instagram, my daughter was really into Instagram, not as much anymore but definitely games. Now it's just like a free-for-all. I mean, they're in their rooms with the door shut, I have no idea what they're doing all day. I just know that they're doing their homework, getting good grades. I think they're on electronics the entire day.Parent 1.0: Yeah, so I have a similar experience with my eight-year-old. Pre-pandemic he was still in second grade so the use of computers was very limited. Maybe once a week he would type, words of the week was one of the options I picked to do. That was his only interaction with computer. Of course he had devices to play on, and he would go to his father's house every other weekend and he would be more exposed there to computers. But still he was busy. He was busy during the week, there was school, there was getting to school, getting from school, that's a trip in itself.Parent 1.0: There was hockey, we had TaeKwonDo, he was very busy. Even then it didn't seem like enough, it didn't seem like we were doing enough but he was certainly a lot busier than now. And he did get to play, mostly probably on the weekends, and again, weekends were also pretty packed. So I don't know how he had time, but he did play, not a lot. And

In [44]:
focus_groups_convo_groupedby_doc_question_df = focus_groups_convo_groupedby_doc_question_df.reset_index()

In [45]:
focus_groups_convo_groupedby_doc_question_df.head()

,focus_group_subtype_id,doc_no_within_subtype,question_id,0
0,1,1,1,"Moderator: So just starting to think through some of the impact of technology, can we talk a little bit about how your child used technology prior to the pandemic for school purposes? Parent 2.0: Oh okay. Well, they didn't use it as much. I mean, the teachers would assign some reading or math assignments on I ready and from time to time some homework. But yeah, that was about it. I would monitor how much time they were on Roblox and other things to entertain themselves, but it was easier control them back then because they didn't have to go on as much. I mean, that's what it was. Parent 3.0: I would say the same thing. My daughter was in fifth grade, we would have a couple in Newsela projects, half an hour of homework. But we were in the playground, neighborhood friends, after school projects, after school swimming. A little bit of tablet on the weekends, but there was no internet access and there was no YouTube in my life, there was no Roblox, there was no gaming. So, a very different world. Parent 5.0: I have a similar experience to Parent 3. My high schooler had a lot of his homework that he had to type and word process... I don't know if that's the right word, into Docs, but it wasn't a huge amount. It wasn't doing research, it wasn't hours and hours. My elementary school guy, he didn't have any homework on the computer at all. And my middle schooler had very little, like type this one thing or something like that. It was very little on the computer. And for myself, in terms of my family, I lock the video games so that there was very little access to video games during the week. Parent 5.0: And then on the weekends there was an hour or something like that, it wasn't that much. There was no YouTube watching, like now YouTube consumes our life. My kids didn't even know how to use the features on our TV, the smart features, they didn't know they could access YouTube on it or anything like that. So it was very controlled and very little. I also had my children in a lot of afterschool activities. My older son was in an orchestra, he also was in a band, theater groups, there was a lot of afterschool activity. A lot of playing outside, a lot of going to the park and all that stuff. Yeah, so... Parent 4.0: I can go. My kids probably had more... It sounds like they did more online than, so far, the rest of them. But my daughter in high school definitely had almost all of her homework... Not all of it because she was assigned in school, but she definitely was on doing homework, I feel like, a good amount. Nothing like now, but definitely was using it. My middle schooler also, but not as much. They both did play games or use Instagram, my daughter was really into Instagram, not as much anymore but definitely games. Now it's just like a free-for-all. I mean, they're in their rooms with the door shut, I have no idea what they're doing all day. I just know that they're doing their homework, getting good grades. I think they're on electronics the entire day. Parent 1.0: Yeah, so I have a similar experience with my eight-year-old. Pre-pandemic he was still in second grade so the use of computers was very limited. Maybe once a week he would type, words of the week was one of the options I picked to do. That was his only interaction with computer. Of course he had devices to play on, and he would go to his father's house every other weekend and he would be more exposed there to computers. But still he was busy. He was busy during the week, there was school, there was getting to school, getting from school, that's a trip in itself. Parent 1.0: There was hockey, we had TaeKwonDo, he was very busy. Even then it didn't seem like enough, it didn't seem like we were doing enough but he was certainly a lot busier than now. And he did get to play, mostly probably on the weekends, and again, weekends were also pretty packed. So I don't know how he had time, but he did play, not a l

In [46]:
#dataframe with sub-convo added
focus_groups_convo_groupedby_doc_question_df = focus_groups_convo_groupedby_doc_question_df.rename(columns={0:'question_convo'})

In [20]:
bert_model = Summarizer()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
gpt2_model = TransformerSummarizer(transformer_type='GPT2', transformer_model_key='gpt2-medium')

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2-medium and are newly initialized: ['h.15.attn.masked_bias', 'h.12.attn.masked_bias', 'h.4.attn.masked_bias', 'h.19.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.17.attn.masked_bias', 'h.18.attn.masked_bias', 'h.22.attn.masked_bias', 'h.14.attn.masked_bias', 'h.1.attn.masked_bias', 'h.13.attn.masked_bias', 'h.8.attn.masked_bias', 'h.6.attn.masked_bias', 'h.11.attn.masked_bias', 'h.5.attn.masked_bias', 'h.21.attn.masked_bias', 'h.7.attn.masked_bias', 'h.16.attn.masked_bias', 'h.10.attn.masked_bias', 'h.23.attn.masked_bias', 'h.0.attn.masked_bias', 'h.9.attn.masked_bias', 'h.20.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
xlnet_model = TransformerSummarizer(transformer_type='XLNet', transformer_model_key='xlnet-base-cased')

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
def get_bert_summary(input_text):
    return bert_model(input_text, min_length = 40)

In [35]:
def get_gpt2_summary(input_text):
    return gpt2_model(input_text, min_length = 40)

In [36]:
def get_xlnet_summary(input_text):
    return xlnet_model(input_text, min_length = 40)

In [48]:
#obtain BERT model summary for each sub-convo
focus_groups_convo_groupedby_doc_question_df['bert_summary'] = focus_groups_convo_groupedby_doc_question_df.apply(lambda x : get_bert_summary(x['question_convo']), axis=1)

In [34]:
focus_groups_convo_groupedby_doc_question_df.head()

,index,focus_group_subtype_id,doc_no_within_subtype,question_id,question_summary,bert_summary
0,0,1,1,1,"Moderator: So just starting to think through some of the impact of technology, can we talk a little bit about how your child used technology prior to the pandemic for school purposes?Parent 2.0: Oh okay. Well, they didn't use it as much. I mean, the teachers would assign some reading or math assignments on I ready and from time to time some homework. But yeah, that was about it. I would monitor how much time they were on Roblox and other things to entertain themselves, but it was easier control them back then because they didn't have to go on as much. I mean, that's what it was.Parent 3.0: I would say the same thing. My daughter was in fifth grade, we would have a couple in Newsela projects, half an hour of homework. But we were in the playground, neighborhood friends, after school projects, after school swimming. A little bit of tablet on the weekends, but there was no internet access and there was no YouTube in my life, there was no Roblox, there was no gaming. So, a very different world.Parent 5.0: I have a similar experience to Parent 3. My high schooler had a lot of his homework that he had to type and word process... I don't know if that's the right word, into Docs, but it wasn't a huge amount. It wasn't doing research, it wasn't hours and hours. My elementary school guy, he didn't have any homework on the computer at all. And my middle schooler had very little, like type this one thing or something like that. It was very little on the computer. And for myself, in terms of my family, I lock the video games so that there was very little access to video games during the week.Parent 5.0: And then on the weekends there was an hour or something like that, it wasn't that much. There was no YouTube watching, like now YouTube consumes our life. My kids didn't even know how to use the features on our TV, the smart features, they didn't know they could access YouTube on it or anything like that. So it was very controlled and very little. I also had my children in a lot of afterschool activities. My older son was in an orchestra, he also was in a band, theater groups, there was a lot of afterschool activity. A lot of playing outside, a lot of going to the park and all that stuff. Yeah, so...Parent 4.0: I can go. My kids probably had more... It sounds like they did more online than, so far, the rest of them. But my daughter in high school definitely had almost all of her homework... Not all of it because she was assigned in school, but she definitely was on doing homework, I feel like, a good amount. Nothing like now, but definitely was using it. My middle schooler also, but not as much. They both did play games or use Instagram, my daughter was really into Instagram, not as much anymore but definitely games. Now it's just like a free-for-all. I mean, they're in their rooms with the door shut, I have no idea what they're doing all day. I just know that they're doing their homework, getting good grades. I think they're on electronics the entire day.Parent 1.0: Yeah, so I have a similar experience with my eight-year-old. Pre-pandemic he was still in second grade so the use of computers was very limited. Maybe once a week he would type, words of the week was one of the options I picked to do. That was his only interaction with computer. Of course he had devices to play on, and he would go to his father's house every other weekend and he would be more exposed there to computers. But still he was busy. He was busy during the week, there was school, there was getting to school, getting from school, that's a trip in itself.Parent 1.0: There was hockey, we had TaeKwonDo, he was very busy. Even then it didn't seem like enough, it didn't seem like we were doing enough but he was certainly a lot busier than now. And he did get to play, mostly probably on the weekends, and again, weekends were also pretty packed. So I don't know how he had t

In [49]:
#obtain GPT2 model summary for each sub-convo
focus_groups_convo_groupedby_doc_question_df['gpt2_summary'] = focus_groups_convo_groupedby_doc_question_df.apply(lambda x : get_gpt2_summary(x['question_convo']), axis=1)

In [50]:
#obtain XLNET model summary for each sub-convo
focus_groups_convo_groupedby_doc_question_df['xlnet_summary'] = focus_groups_convo_groupedby_doc_question_df.apply(lambda x : get_xlnet_summary(x['question_convo']), axis=1)

In [51]:
focus_groups_convo_groupedby_doc_question_df.to_csv('pretrained_model_summaries_1.csv')

### Merge Sort on these summaries would result in the final combined summary
#### Merge sort would help in preserving the text order, of the differences from each model, as found in the original text whose summary is required and thus would preserve context and make the summaries more meaningful 